<a href="https://colab.research.google.com/github/robgon-art/InventorBot/blob/main/InventorBot_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install keybert
from keybert import KeyBERT
model = KeyBERT('distilbert-base-nli-mean-tokens')
doc = "Method and apparatus for parking lot metering using activation codes"
keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=1)
print(keywords[0])
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
from datetime import date, timedelta
import requests

num_years = 45
the_start = 1976

for start_year in range(the_start, the_start + num_years):
  for month in range(1, 13):
    print("get patents", start_year, month)
    start_date = date(start_year, month, 1)
    if month < 12:
      end_date = date(start_year, month+1, 1)
    else:
      end_date = date(start_year+1, 1, 1)    
    start = start_date
    end = start + timedelta(weeks=1)
    f = open("/content/drive/MyDrive/patents2/patents_" + start.strftime('%Y-%m-%d') + ".txt", "w")
    while start < end_date:
      if end > end_date:
        end = end_date
      print("Getting patents for", start.strftime('%Y-%m-%d'), "to", end.strftime('%Y-%m-%d'))

      resp = requests.get('https://api.patentsview.org/patents/query?' + 
                          'q={"_and":[{"_gte":{"patent_date":"' + start.strftime('%Y-%m-%d') + '"}},' +
                          '{"_lt":{"patent_date":"' +  end.strftime('%Y-%m-%d') + '"}}]}' +
                          '&f=["patent_title","patent_abstract"]&o={"page":1,"per_page":10000}')
      
      if resp.json()["patents"] is None:
        break

      for patent in resp.json()["patents"]:
        if patent["patent_abstract"] is not None:
          title = patent["patent_title"]
          keywords = model.extract_keywords(title, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=1)
          if (len(keywords) == 0):
            keywords = title.lower()
          f.write("KEYWORDS: " + keywords[0] + "\n")
          f.write("TITLE: " + title + "\n")
          abstract = patent["patent_abstract"]
          sentences = tokenizer.tokenize(abstract)
          short_abstract = " ".join(sentences[0:2])
          f.write("ABSTRACT: " + short_abstract + "\n")
          f.write("***\n")
          count += 1
      print(end.strftime('%Y-%m-%d'), resp.json()["total_patent_count"], "total patents:", f'{count:>10}')
      start += timedelta(weeks=1)
      end += timedelta(weeks=1)
  f.close()